In [1]:
import os, random, logging
import utils.auxua as aux
import utils.watch as watch
import utils.tf_formh5 as tf_formh5
import numpy as np

''' tf log level & neptune'''
tf_formh5.set_tf_loglevel(logging.ERROR)

cudatoolkit               10.1.243            h8cb64d8_11    conda-forge
cudnn                     7.6.5.32             hc0a50b0_1    conda-forge
neptune-tensorflow-keras  2.1.0              pyhd8ed1ab_0    conda-forge
numpy                     1.19.5           py38hc896f84_4  
numpy-base                1.19.5           py38h21a3de8_4  
tensorflow                2.2.0           gpu_py38hb782248_0    anaconda
tensorflow-base           2.2.0           gpu_py38h83e3d50_0    anaconda
tensorflow-estimator      2.6.0            py38h709712a_0    conda-forge
tensorflow-gpu            2.2.0                h0d30ee6_0    anaconda

Num GPUs Available:  4


In [2]:
'''get_results_from_txt'''

res_list_full,res_list_max,res_list_fn,res_list_path,res_list_labels,res_path,res_model_fn,res_model_fn_strap =\
aux.get_results_from_txt(aux.RSLT_PATH,printt=True)

res_list={
    'full':res_list_full,
    'max':res_list_max,
    'videoname':res_list_fn,
    'videopath':res_list_path,
    'labels':res_list_labels,
    'txt_path':res_path,
    'txt_modelname':res_model_fn,
    'txt_modelname_strap':res_model_fn_strap        
}

for i in range(len(res_list['txt_modelname'])):print(i,res_list['txt_modelname'][i])

#res_list_1 = aux.get_results_from_txt("/raid/DATASETS/.zuble/vigia/zu++/model/rslt/1671124367.446378_3gelu_adamamsgrad_0_4000_weights.txt")

FOLDER

results for 3gelu_sgd_0_4000_weights
	ACCURACY  0.8187
	PRECISION (% of True 1 out of all Positive predicted) 0.8867
	RECALL (%% of True Positive out of all actual anomalies)  0.814
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9295 
	 AUC-ROC 0.8896 
	F1_SCORE (harmonic mean of precision and recall)  0.8488 0.8488 

results for 3gelu_adam_0_4000_weights
	ACCURACY  0.7712
	PRECISION (% of True 1 out of all Positive predicted) 0.9117
	RECALL (%% of True Positive out of all actual anomalies)  0.702
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9290 
	 AUC-ROC 0.8835 
	F1_SCORE (harmonic mean of precision and recall)  0.7932 0.7932 

results for 3gelu_adamamsgrad_0_4000_weights
	ACCURACY  0.7875
	PRECISION (% of True 1 out of all Positive predicted) 0.9167
	RECALL (%% of True Positive out of all actual anomalies)  0.726
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9398 
	 AUC-ROC 0.9044 
	F1_SCORE (harmonic mean of precision and recall)  0.8103 0.8103 

results for leakyrelu_adam_0_2000_w

In [5]:
def init_watch_reslist(res_list, txt_i,watch_this):
    
    def get_fn_index_from_list(txt_i):
        '''retrives all video indexs from a rslt.txt that are false negatives'''
        
        print("\nget_fn_index_from_list ",res_list['txt_modelname'][txt_i])

        labels_indexs_fn={'AA':[],'B1':[],'B2':[],'B4':[],'B5':[],'B6':[],'G':[],'FNB1':[],'FNB2':[],'FNB4':[],'FNB5':[],'FNB6':[],'FNG':[],'FNBG':[]}


        for video_j in range(len(res_list['videoname'])):
            if 'label_A' not in res_list['videoname'][video_j][txt_i]: #its anomalie
                labels_indexs_fn['AA'].append(video_j)
                if float(res_list['max'][video_j][txt_i])<=0.5: #FN
                    labels_indexs_fn['FNBG'].append(video_j) 
                    if 'label_B1' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['FNB1'].append(video_j)
                    if 'label_B2' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['FNB2'].append(video_j)
                    if 'label_B4' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['FNB4'].append(video_j)
                    if 'label_B5' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['FNB5'].append(video_j)
                    if 'label_B6' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['FNB6'].append(video_j)
                    if 'label_G'  in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['FNG'].append(video_j)
                else: #TRUE POSITIVE
                    if 'label_B1' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B1'].append(video_j)
                    if 'label_B2' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B2'].append(video_j)
                    if 'label_B4' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B4'].append(video_j)
                    if 'label_B5' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B5'].append(video_j)
                    if 'label_B6' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B6'].append(video_j)
                    if 'label_G'  in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['G'].append(video_j)
                    
        return labels_indexs_fn
    
    as_total = watch.get_as_total_from_res_list(res_list,txt_i)
    
    labels_indexs_fn=get_fn_index_from_list(txt_i)
    print(  '\nFALSE NEGATIVES',\
            '\n  B1 FIGHT',  len(labels_indexs_fn['FNB1']),\
            '\n  B2 SHOOT',  len(labels_indexs_fn['FNB2']),\
            '\n  B4 RIOT',   len(labels_indexs_fn['FNB4']),\
            '\n  B5 ABUSE',  len(labels_indexs_fn['FNB5']),\
            '\n  B6 CARACC', len(labels_indexs_fn['FNB6']),\
            '\n  G EXPLOS',len(labels_indexs_fn['FNG']),\
            '\n  B+G',len(labels_indexs_fn['FNBG']),\
            '\nANOMALIES',\
            '\n  B1 FIGHT',  len(labels_indexs_fn['B1']),\
            '\n  B2 SHOOT',  len(labels_indexs_fn['B2']),\
            '\n  B4 RIOT',   len(labels_indexs_fn['B4']),\
            '\n  B5 ABUSE',  len(labels_indexs_fn['B5']),\
            '\n  B6 CARACC', len(labels_indexs_fn['B6']),\
            '\n  G EXPLOS',len(labels_indexs_fn['G']),\
            '\n\nALL ANOMALIES',len(labels_indexs_fn['AA']),'\n')
    
    print('set to watch this',watch_this)
    for labels_2_watch in watch_this[1:]:
        print(labels_2_watch)
        for i in range(len(labels_indexs_fn[labels_2_watch])):
            video_j = labels_indexs_fn[labels_2_watch][i]
            print(video_j)
            #watch.cv2_test_from_aslist(res_list,as_total[video_j],video_j,txt_i)

In [6]:
'''
FN false negatives
AA all anomalies

['FN or AA' , 'AA'] : ALL
['FN or AA' , 'B1'] : FIGHT
['FN or AA' , 'B2'] : SHOOTING
['FN or AA' , 'B4'] : RIOT
['FN or AA' , 'B5'] : ABUSE
['FN or AA' , 'B6'] : CAR ACCIDENT
['FN or AA' , 'G '] : EXPLOSION 
['FN      ' , 'BG'] : All FN 

'''

init_watch_reslist(res_list,9,watch_this=['AA','B5'])

as_total from /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677498953.2416248_relu_sgd_0_4000_weights.txt

get_fn_index_from_list  1677498953.2416248_relu_sgd_0_4000_weights

FALSE NEGATIVES 
  B1 FIGHT 27 
  B2 SHOOT 10 
  B4 RIOT 16 
  B5 ABUSE 4 
  B6 CARACC 6 
  G EXPLOS 5 
  B+G 68 
ANOMALIES 
  B1 FIGHT 93 
  B2 SHOOT 74 
  B4 RIOT 85 
  B5 ABUSE 4 
  B6 CARACC 90 
  G EXPLOS 86 

ALL ANOMALIES 500 

set to watch this ['AA', 'B5']
B5
75
262
284
787
